<a href="https://colab.research.google.com/github/HirunaD/LangChain/blob/main/01_Simple_Chatbot_Using_LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install the necessary packages
!pip install langchain -qU
!pip install langchain-openai -qU

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.2/65.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.1/438.1 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 kB 16.4 MB/s eta 0:00:00


In [2]:
# Import necessary libraries
import os
from google.colab import userdata

**Initialize OpenAI LLM**

In [3]:
from langchain_openai import ChatOpenAI

# Set OpenAI API key
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

# Initialize the ChatOpenAI model
llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0
)

**Initialize Prompt Template**

In [4]:
from langchain_core.prompts import ChatPromptTemplate

# Create a prompt template
prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are an intelligent chatbot. Answer the following question."),
        ("user", "{question}")
    ]
)

**Initialize Output Parser**

In [5]:
from langchain_core.output_parsers import StrOutputParser

# Initialize the string output parser
parser = StrOutputParser()

In [6]:
# Chain the prompt, LLM, and output parser
chain = prompt | llm | parser

In [7]:
question = "My name is Hiruna De Silva"

response = chain.invoke({"question": question})

print(response)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}